In [26]:
import pandas as pd
import numpy as np
import tabula
import math
import os
from datetime import date

In [27]:
#este bloque busca los BOD en PDF en la carpeta de documentos (docs_dir), los compara con los documentos que ya se procesaron en su día guardados como .pkl (pkld_list) y genera una lista
#de nombres de archivos a procesar (proc_docs)

docs_dir = 'D:/jaume/Datasets/BOD/'
pkld_dir = 'D:/jaume/Jupyter Notebooks/Vacantes Scraper/ScrapedData/'

docs_list = []    #lista de documentos en la carpeta de entrada
pkld_list = []    #lista de documentos ya procesados y transformados a pkl
proc_docs = []    #lista de documentos aún por procesar

# iterate over files in that directory
for filename in os.scandir(docs_dir):
    if filename.is_file():
        docs_list.append(filename.name.split('_')[1])
        
# iterate over files in that directory
for filename in os.scandir(pkld_dir):
    if filename.is_file():
        pkld_list.append(filename.name.split('.')[0])

#seriales de documentos a procesar
pending_list = [doc for doc in docs_list if doc not in pkld_list]

# list of files to porcess
for filename in os.scandir(docs_dir):
    if filename.name.split('_')[1] in pending_list:
        proc_docs.append(filename)

In [28]:
for doc in proc_docs:
    
    print('Processing ' + doc.name + '...')
    file_pdf = docs_dir + doc.name
    read_pdf = tabula.read_pdf(file_pdf, pages = 'all', silent = True)
    
    sel_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (14-16) y el número de filas (2)
    oth_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (14-16) y el más de 2 filas (funcionan diferente)
    for i in range(len(read_pdf)):
        #seleccionamos sólo los df que contienen de 14 a 16 columnas y 2 filas
        if np.logical_and(read_pdf[i].shape[0] == 2, np.logical_and(read_pdf[i].shape[1] >= 14, read_pdf[i].shape[1] <= 16)):
            sel_tables.append(i)

        
        #seleccionamos sólo los df que contienen más de dos filas, no contienen "voluntario o forzoso" (esas son vacantes) y tienen de 14 a 16 columnas:
        elif np.logical_and(
            np.logical_and(~read_pdf[i].isin(['Servicio Activo']).any().any(),
                           np.logical_and(read_pdf[i].shape[0] > 2, ~read_pdf[i].isin(['VOLUNTARIO', 'FORZOSO', 'VOL', 'FOR']).any().any())),
            np.logical_and(read_pdf[i].shape[1] >= 14, read_pdf[i].shape[1] <= 16)):
            oth_tables.append(i)    

        #seleccionamos los destinos
        elif read_pdf[i].isin(['VOLUNTARIO', 'FORZOSO', 'VOL', 'FOR']).any().any():
            pass    #aquí FALTA incluir el código para crear una lista de DF de DESTINOS
            
        else:
            continue

    df_list = [read_pdf[index] for index in sel_tables]    #conservará los elementos de read_tables seleccionados en sel_tables
    oth_list = [read_pdf[index] for index in oth_tables]    #conservará los elementos de read_tables seleccionados en sel_tables

    df_clean = []
    oth_clean = []
    errors = []

    #código para los df de shape (2, 14-16) almacenados en df_list

    for i in range(len(df_list)):
        df = df_list[i]                                   #iteramos los df seleccionados en df_list
        col_list =[]                                      #instanciamos la lista de columnas vacía que se poblará con los 'split' de cada columna del df de esta iteración
        string = [np.nan] * df.shape[1]
        col = [np.nan] * df.shape[1]
        
        if isinstance(df.iloc[1, 0], str):                    #n_vacs define el número de vacantes que se esperan (num de vacantes separadas por \r en la 1a col del DF)
            n_vacs = len(df.iloc[1, 0].split('\r'))
            
        elif isinstance(df.iloc[1, 0], float):
            n_vacs = 1
            
        else:
            print('DF-' + str(i) + 'found a ' + str(type(df.iloc[1, 0])) + ' in iloc [0, 1]')

        for j in range(df.shape[1]):                      #iteramos cada columna (str separado por '\r') del df para convertirlo en una lista de valores de la columna

            string[j] = df.iloc[1, j]

            if type(string[j]) == str:

                col[j] = string[j].split('\r')            #columna resultante de la separación de 'string'

                if len(col[j]) == n_vacs:
                    col_list.append(col[j])               #si la columna tiene exactamente el mismo número de registros que el número de vacantes del df, adjuntamos la columna

                elif math.ceil(len(col[j])/2) == n_vacs:
                    col_list.append(col[j][0::2])         #si la columna tiene el doble (redondeado) de registros que el número de vacantes del df, adjuntamos la columna cada dos espacios

                else:                                     #si no tiene el mismo número, lanzamos la lógica para colocar todos los valores posibles en ese registro y marcamos el error
                    errors.append('DF-' + str(i) + ' Elementos de la columna ' + str(j) + ' no definidos en todas las vacantes. Ver opciones.')
                    unique = set(col[j])
                    values = list(unique)
                    values.append(np.nan)
                    col_list.append([values] * n_vacs)
            
            elif type(string[j]) == np.float64:
                col[j] = string[j]
                col_list.append(col[j])
                
            else:
                col_list.append([np.nan] * n_vacs)

        col_map = {}
        if np.logical_and(len(col_list) == 14, np.logical_and('PA' not in col_list[5], 'LD' not in col_list[5])):                             #mapeo según resoluciones de 14 columnas
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'ta': col_list[5],
                       'empleo': col_list[6],
                       'efun': col_list[7],
                       'cursos': col_list[8],
                       't_max': col_list[9],
                       't_min': col_list[10],
                       'fecha_cob': col_list[11],
                       'csce': col_list[12],
                       'obs': col_list[13]
                      }
            
        elif np.logical_and(len(col_list) == 14, np.logical_or('PA' in col_list[5], 'LD' in col_list[5])):                             #mapeo según resoluciones de 14 columnas
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[5],
                       'ta': col_list[6],
                       'empleo': col_list[7],
                       'efun': col_list[8],
                       'cursos': col_list[10],
                       'csce': col_list[12],
                       'obs': col_list[13],
                       'cantidad_vacantes': col_list[4]
                      }

        elif np.logical_and(len(col_list) == 15, 'CM' in col_list[4]):                           #mapeo según resoluciones de 15 columnas agregar condición 'y en la columna 4 contiene CM' para
            col_map = {'n_vac': col_list[0],                                                     #deconflictar con vacantes de RESERVA
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'ta': col_list[5],
                       'empleo': col_list[6],
                       'efun': col_list[7],
                       'cursos': col_list[8],
                       't_max': col_list[9],
                       't_min': col_list[10],
                       'fecha_cob': col_list[11],
                       'csce': col_list[12],
                       'cod_cm': col_list[13],
                       'obs': col_list[14]
                      }
            
        elif np.logical_and(len(col_list) == 15, 'CM' not in col_list[4]):                           #mapeo según resoluciones de 15 columnas diferentes a CM (pendiente mapeo)
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'empleo': col_list[7],
                       'csce': col_list[13],
                       'obs': col_list[14]
                      }

        elif len(col_list) == 16:                           #mapeo según resoluciones de 16 columnas
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'ta': col_list[5],
                       'ejercito': col_list[6],
                       'cuerpo_esc': col_list[7],
                       'empleo': col_list[8],
                       'efun': col_list[9],
                       'cursos': col_list[10],
                       't_max': col_list[11],
                       't_min': col_list[12],
                       'nivel': col_list[13],   #comprobar que esto es así siempre.
                       'csce': col_list[14],
                       'obs': col_list[15]
                      }

        else:
            errors.append('DF-' + str(i) + ' has wrong COL MAPPING')


        try:
            if ~pd.DataFrame(col_map).isin(['VOLUNTARIO', 'FORZOSO']).any().any():    #comprobamos que no son DESTINOS. esta característica los distingue.
                data = pd.DataFrame(col_map)
                df_clean.append(data)
            else: pass

        except:
            errors.append('DF-' + str(i) + ' threw an error in DataFrame')



    print(str(len(df_clean)) + ' clean df,s in the list, ready to pack')

    #código para los df de shape (2+, 14-16) almacenados en oth_list

    for oth_df in oth_list:

        oth_df = oth_df.iloc[1::2]

        vac_uco = oth_df.iloc[:, 0].str.split(n = 1, expand = True)
        
        if vac_uco.shape[1] == 2:                                     #si la línea para crear vac_uco genera dos columnas (n_vac y UCO) el código es bueno y sigue.
            n_vac = vac_uco.iloc[:, 0]
            uco = vac_uco.iloc[:, 1]    
        
        else:                                                         #si no crea dos columnas es un error, el DF no me sirve, saltamos al siguiente.
            continue
        
        
        
        clas_ta = oth_df['CLAS.'].str.split(n = 1, expand = True)    
        asig = clas_ta.iloc[:, 0]
        ta = clas_ta.iloc[:, 1]
        
        obs = oth_df.iloc[:, -1]

        assert len(n_vac) == len(uco) == len(asig) == len(ta)        #checkea que hay tantos n_vac como ucos, modos de asignación y TAs
        
        pt_ind = 0
        ciu_ind = 0
        cm_ind = 0
        csce_ind = 0
        empleo_ind = 0

        for i in range(oth_df.shape[1]):

                if all([isinstance(e, str) for e in oth_df.iloc[:, i]]):
                    if round(oth_df.iloc[:, i].str.len().mean()) == 8:
                        ciu_ind = i
  
                    elif oth_df.iloc[:, i].str.count('/').sum() == len(oth_df.iloc[:, i]):
                        pt_ind = i
   
                    elif np.logical_and(oth_df.iloc[:, i].str.len().mean() == 4, ~oth_df.iloc[:, i].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').all()):
                        cm_ind = i
          
                    elif oth_df.iloc[:, i].str.len().mean() == 6:
                        csce_ind = i
            
                    elif oth_df.iloc[:, i].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').sum() == len(oth_df.iloc[:, i]):
                        empleo_ind = i
                        
                    else: pass
        

        col_map = {'n_vac': n_vac,
               'uco': uco,
               'ciu': oth_df.iloc[:, ciu_ind],
               'pt': oth_df.iloc[:, pt_ind],
               'asig': asig,
               'ta': ta,
               'empleo': oth_df.iloc[:, empleo_ind],
    #           'efun': col_list[7],
    #           'cursos': col_list[8],
    #           't_max': col_list[9],
    #           't_min': col_list[10],
    #           'fecha_cob': col_list[11],
               'csce': oth_df.iloc[:, csce_ind],
               'cod_cm': oth_df.iloc[:, cm_ind],
               'obs': obs
              }


        data = pd.DataFrame(col_map)
        oth_clean.append(data)
        errors.append('DF_OTH-' + str(i) + ' threw an error in DataFrame')

    print(str(len(df_clean)+len(oth_clean)) + ' clean df,s in 2 lists, ready to pack')

    pkl_name_parts = file_pdf.split('_')

    try:
        df_full = pd.concat(df_clean + oth_clean).reset_index(drop = True)      #la solución!!!
        df_full['fecha_pub'] = pkl_name_parts[1]
    except: pass
    
    pkl_name = pkl_name_parts[1] + '.pkl'

    df_full.to_pickle(pkld_dir + pkl_name)

    print(str(pkl_name_parts[1]) + '_' + str(pkl_name_parts[2]) + ' pickled!')

Processing BOD_20220125_16.pdf...
2 clean df,s in the list, ready to pack
3 clean df,s in 2 lists, ready to pack
20220125_16.pdf pickled!


In [29]:
errors

['DF-0 Elementos de la columna 7 no definidos en todas las vacantes. Ver opciones.',
 'DF-1 Elementos de la columna 3 no definidos en todas las vacantes. Ver opciones.',
 'DF-1 Elementos de la columna 7 no definidos en todas las vacantes. Ver opciones.',
 'DF_OTH-14 threw an error in DataFrame']

In [36]:
oth_list[0]

,NUM.\rVAC.,"DIRECCION GENERAL, ORGANISMO, CENTRO, UNIDAD,\rLOCALIDAD",CODIGO\rIDENTIF.\rUNIDAD,CODIGO\rPUESTO\rTRABAJO,CLAS.,EMPLEO,ESPEC. FUNDAMENTAL\rESPEC. COMPLEMENTARIA,TITUL./SEG. ESP. EXIGIBLES\rTITUL./SEG. ESP. VALORABLES,TIEMPOS,FECHA\rCOBERTURA,CSCE\rEUROS/MES,TIPO\rCM,OBSERVACIONES,Unnamed: 0,Unnamed: 1
0,NaN,NaN,NaN,AS.,TA,MAX.,MIN.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00200\r00201\r00202\r00203\r00204\r00205\r0020...,AGRUPACION DE TRANSPORTE NUM.1\rMADRID\rCUARTE...,50022962\r50030001\r50001986\r57005389\r572948...,5JA39/002\r5JA39/012\r5JA39/006\r5JA39/004\r5J...,CM\rCM\rCM\rCM\rCM\rCM\rCM\rCM\rCM\rCM,C\rC\rC\rC\rC\rC\rC\rC\rC\rC,CTE\rCTE\rCTE\rCTE\rCTE\rCTE\rCTE\rCTE\rCTE\rCTE,NaN,"SL:I, RV:I, QM:I, GL:I\rSL:I, RV:I, QM:I, GL:I...",10\r10\r10\r10\r10\r10\r10\r10\r10\r10,3\r3\r3,NaN,"499,78\r499,78\r499,78\r499,78\r382,37\r499,78...",CM17\rCM17\rCM17\rCM21\rCM17\rCM21\rCM25\rCM25...,"244,511,959\r244,511,656,960\r244,511,656,960\..."
2,00210,BETERA (VALENCIA)\rCUARTEL GENERAL DEL MANDO D...,50085418,5JA48/002,CM,C,CTE,TRA.,TI:E,10,NaN,NaN,"540,28",CM15,"244,511,656,960"


In [38]:
df_full.to_csv('tst.csv')